In [2]:
import torch
import torch.utils.data
import numpy as np
from matplotlib import pyplot as plt
import PIL
import albumentations
import pretrainedmodels
from IPython.core.debugger import set_trace
from tqdm import tqdm_notebook
import skimage
import pandas as pd
from sklearn.model_selection import StratifiedKFold

from nb_006 import *

from pathlib import Path

PATH = Path('../data')
TRAIN_DN = 'train'
TEST_DN = 'test'
IMAGES = 'images'
MASKS = 'masks'

DEPTHS_FN = 'depths.csv'

In [5]:
imagenet_means, imagenet_std = map(np.array, ([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]))

normalize = lambda x: (x - imagenet_means) / imagenet_std
denormalize = lambda x: x * imagenet_std + imagenet_means

def open_image(fn):
    x = PIL.Image.open(fn).convert('RGB')
    return normalize(np.asarray(x) / 255)

def open_mask(fn):
    x = PIL.Image.open(fn)
    return np.asarray(x) / 65535

def show_image(im, figsize=None, ax=None, alpha=None):
    if im.shape[0] == 3: im = im.transpose(1,2,0)
    if im.min() < 0 and im.ndim == 3: im=denormalize(im); im = np.clip(im, 0, 1) # this is quite horrible and can lead to bugs
    if not ax: fig,ax = plt.subplots(figsize=figsize)
    ax.imshow(im, alpha=alpha)
    ax.set_axis_off()
    return ax

class FilesDataset(DatasetBase):
    def __init__(self, folder, sz, take_idxs=None, tfms=None):
        sub_folders = list(p.stem for p in (folder/str(sz)).iterdir())
        self.x = list((folder/str(sz)/'images').iterdir())
        if take_idxs is not None:
            self.x = [self.x[idx] for idx in take_idxs]
        if 'masks' in sub_folders:
            self.y = list((folder/str(sz)/'masks').iterdir())
            if take_idxs is not None:
                self.y = [self.y[idx] for idx in take_idxs]
            assert np.all([p1.stem == p2.stem for p1, p2 in zip(self.x, self.y)]) and len(self.x) == len(self.y), \
                'filenames between self.x and self.y do not match'
        self.tfms = tfms
            
    def __getitem__(self,i):
        im = open_image(self.x[i])
        if hasattr(self, 'y'):
            mask = open_mask(self.y[i])
        else:
            mask = np.zeros_like(im)[0]
        if self.tfms is not None:
            transformed = self.tfms(image=im, mask=mask)
            im, mask = transformed['image'], transformed['mask']        
        im = im.transpose(2, 0, 1)
        im, mask = map(lambda x: x.astype(np.float32), [im, mask])
        return im, mask
    
    def show_im_with_mask(self, idx, axes=None):
        im, mask = self[idx]
        if axes is None: _, axes = plt.subplots(1, 2)
        show_image(im, ax=axes[0])
        show_image(mask, ax=axes[1])
        
    def check_tfms(self, idx, n=5, figsize=(10,4)):
        _, axes = plt.subplots(2, n, figsize=figsize)
        for i in range(n):
            self.show_im_with_mask(idx, axes=(axes[0][i], axes[1][i]))

n = len(list((PATH/TRAIN_DN/'128/images').iterdir()))

In [ ]:
# np.random.seed(0)
# idxs = np.arange(n)
# np.random.shuffle(idxs)

# val_parts = [idxs[i*800:(i+1)*800] for i in range(5)]
# trn_parts = [[idx for idx in idxs if idx not in val_parts[i]] for i in range(5)]

In [48]:
# mask_sizes = {}
# for p in (PATH/TRAIN_DN/'128/masks').iterdir():
#     img = PIL.Image.open(p)
#     mask_sizes[p.stem] = np.sum(np.asarray(img) != 0)

# pd.to_pickle(mask_sizes, PATH/'mask_sizes.pkl')

In [168]:
# mask_sizes = pd.read_pickle(PATH/'mask_sizes.pkl')

# depths = pd.read_csv(PATH/DEPTHS_FN)
# depths.set_index('id',drop=True, inplace=True)
# depths = depths.to_dict()['z']

# fns = [p.stem for p in (PATH/TRAIN_DN/'128/images').iterdir()]
# trn_df = pd.DataFrame(data={'fn': fns})
# trn_df['coverage'] = trn_df['fn'].apply(lambda x: mask_sizes[x])
# trn_df.coverage /= trn_df.coverage.max()

# def cov_class(v):
#     for i in range(9, -1, -1):
#         if v * 10 >= i: return i

# trn_df['cov_class'] = trn_df.coverage.apply(lambda x: cov_class(x))

# trn_df.to_csv(PATH/'trn_df_with_cov.csv', index=False)

In [169]:
trn_df = pd.read_csv(PATH/'trn_df_with_cov.csv')

In [171]:
trn_df.head()

,fn,coverage,cov_class
0,a7a124fff5,0.980712,9
1,8620f40403,0.210218,2
2,ae11c7b131,0.529207,5
3,57293a65d5,0.731612,7
4,d69c382be5,0.083257,0


In [188]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

In [195]:
parts = list(skf.split(np.zeros(n), trn_df.cov_class))

In [196]:
val_parts = [parts[i][1] for i in range(5)]
trn_parts = [parts[i][0] for i in range(5)]

In [8]:
# np.random.seed(0)
# idxs = np.arange(n)
# np.random.shuffle(idxs)

# val_parts = [idxs[i*800:(i+1)*800] for i in range(5)]
# trn_parts = [[idx for idx in idxs if idx not in val_parts[i]] for i in range(5)]

@dataclass()
class DataBunch():
    train_dl:DataLoader
    valid_dl:DataLoader
    test_dl:DataLoader
    device:torch.device=None

# def get_data_bunch(sz=128, bs=64, part=0, trn_tfms=None):
#     trn_ds = FilesDataset(PATH/TRAIN_DN, sz, take_idxs=trn_parts[part], tfms=trn_tfms)
#     val_ds = FilesDataset(PATH/TRAIN_DN, sz, take_idxs=val_parts[part])
#     tst_ds = FilesDataset(PATH/TEST_DN, sz)
    
#     trn_dl = DataLoader(trn_ds, bs, True, num_workers=11, pin_memory=True)
#     val_dl = DataLoader(val_ds, 2*bs, False, num_workers=11, pin_memory=True)
#     tst_dl = DataLoader(tst_ds, 2*bs, False, num_workers=11, pin_memory=True)
    
#     trn_dl, val_dl, tst_dl = map(lambda ts: DeviceDataLoader(*ts), zip([trn_dl, val_dl, tst_dl], [default_device] * 3) )
#     return DataBunch(trn_dl, val_dl, tst_dl, default_device)

def accuracy_thresh(out, yb, thresh=0.5):
    preds = torch.sigmoid(out) > thresh
    return (preds==yb.byte()).float().mean()

def dice(pred, targs):
    pred = (pred>0).float()
    return 2. * (pred*targs).sum() / (pred+targs).sum()

def predict(model, dl, flip=True):
    model.eval()
    return np.concatenate([torch.sigmoid(model(xb)).detach().cpu().numpy() for xb, yb in dl])

# def predict_with_targs(model, dl, flip=True):
#     model.eval()
#     preds_targs = [(model(xb).detach(), yb.detach().cpu().numpy()) for xb, yb in dl]
#     preds, targs = list(zip(*preds_targs))
#     if not flip:
#         preds = [torch.sigmoid(pred).cpu().numpy() for pred in preds]
#         return np.concatenate(preds), np.concatenate(targs)
#     flipped_preds = [model(torch.flip(xb, [3])).detach() for xb, yb in dl]
#     unflipped_preds = torch.flip(torch.cat(flipped_preds), [2])
#     preds = (torch.cat(preds) + unflipped_preds) / 2
#     return preds.sigmoid().cpu().numpy(), np.concatenate(targs)

def predict_with_targs(model, dl, flip=True):
    model.eval()
    preds_targs = [(torch.sigmoid(model(xb)).detach().cpu().numpy(), yb.detach().cpu().numpy()) for xb, yb in dl]
    preds, targs = list(zip(*preds_targs))
    preds, targs = np.concatenate(preds), np.concatenate(targs)
    if not flip: return preds
    flipped_preds = np.concatenate([torch.sigmoid(model(torch.flip(xb, [3]))).detach().cpu().numpy() for xb, yb in dl])
    preds = (preds + flipped_preds[:,:,::-1]) / 2
    return preds, targs

def accuracy_np(preds, targs): return np.mean((preds > 0.5) == targs)

# https://www.kaggle.com/divrikwicky/u-net-with-simple-resnet-blocks-forked
iou_thresholds = np.array([0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95])

def iou(img_true, img_pred):
    i = np.sum((img_true*img_pred) >0)
    u = np.sum((img_true + img_pred) >0)
    if u == 0:
        return u
    return i/u

def iou_metric(imgs_true, imgs_pred):
    num_images = len(imgs_true)
    scores = np.zeros(num_images)
    
    for i in range(num_images):
        if imgs_true[i].sum() == imgs_pred[i].sum() == 0:
            scores[i] = 1
        else:
            scores[i] = (iou_thresholds <= iou(imgs_true[i], imgs_pred[i])).mean()
            
    return scores.mean()

def filter_image(img, mask_t=100):
    if img.sum() < mask_t:
        return np.zeros(img.shape)
    else:
        return img

# https://www.kaggle.com/paulorzp/run-length-encode-and-decode
def rle_encode(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels = img.flatten(order='F')
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

# def rle_encode(im):
#     pixels = im.flatten(order='F')
#     pixels = np.concatenate([[0], pixels, [0]])
#     runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
#     runs[1::2] -= runs[::2]
#     return ' '.join(str(x) for x in runs)

 
def rle_decode(mask_rle, shape):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)

def preds_to_sub(preds, paths, sig_t, mask_t, name):
    fns = []
    rles = []
    for path, pred in zip(paths, preds):
        pred = pred > sig_t
        pred = filter_image(pred, mask_t)
        fns.append(path.stem)

        resized = skimage.transform.resize(pred.astype(np.uint8) * 255, (101, 101), order=0, mode='constant', anti_aliasing=False)
        rles.append(rle_encode(resized))
    pd.DataFrame(data={'id': fns, 'rle_mask': rles}).to_csv(f'../subs/{name}.csv.gz', compression='gzip', index=False)